In [30]:
import sys
!{sys.executable} -m pip install plac

from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
!{sys.executable} -m spacy download en
from tqdm import tqdm 
nlp1 = spacy.load('en_core_web_sm')

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 12.8 MB 387 kB/s eta 0:00:01    |█▊                              | 696 kB 791 kB/s eta 0:00:16
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Train Data

In [66]:
TRAIN_DATA = [
    ('Pep Guardiola has won a Treble and I believe he can win another one in the next 2 years with Man City', {
        'entities': [(0,13,'PERSON'),(24,30,'EVENT'),(70,87,'DATE'),(93,101,'ORG')]
    }),
    ('Yesterday, on June 13, 2023, I attended the FIFA World Cup final in the Maracanã Stadium, Rio de Janeiro. The match was intense, and Brazil emerged as the champions, winning 2-1 against Argentina.', {
        'entities': [(0,9,'DATE'),(14,27,'DATE'),(44,64,'EVENT'),(72,88,'FAC'),(90,104, 'LOC'),(133, 124, 'FAC'),(133, 139, 'GPE'),(174, 174, 'CARDINAL'),(176, 177, 'CARDINAL'),(186, 196, 'GPE')]
    }),
    ("I bought 5 apples at the grocery store.", {
        'entities': [(9, 10, 'CARDINAL'), (11, 17, 'PRODUCT')]
    }),
    ("The meeting is scheduled for tomorrow.", {
        'entities': [(29, 37, 'DATE')]
    }),
    ("The Kevin Hart Show was held at Madison Square Garden.", {
        'entities': [(4, 14, 'PERSON'), (32, 53, 'LOC')]
    }),
    ("I visited Paris, the city of love.", {
        'entities': [(10, 15, 'GPE')]
    }),
    ("He speaks fluent French and Spanish.", {
        'entities': [(17, 23,'LANGUAGE'), (28, 35,'LANGUAGE')]
    }),
    ('Don Khoa loves Luyi Han more than anything', {
        'entities': [(0,8,'PERSON'), (15,23,'PERSON')]
    }),
    ('Elon Musk purchased twitter for 45 billion dollar.', {
        'entities': [(0,9,'PERSON'),(20,27,'ORG'),(32,49,'MONEY')]
    }),
    ('The first time I heard about the Second Amendment was my first conversation with friends at the University of Rochester.', {
        'entities': [(4,9,'ORDINAL'),(29,49,'LAW'),(92,119,'ORG')]
    }),
    ('Anthony Son Bao had 4 legs but lost one just now cause he touched Don Khoa.', {
        'entities': [(0,15,'PERSON'),(66,74,'PERSON')]
    }),
    ('The iPhone X is a popular Apple product known for its sleek design and advanced features.',
     { 'entities': [(4,12,'PRODUCT'),(26,31,'ORG')]}
    ),
    ('"The Mona Lisa" is a renowned work of art painted by Leonardo da Vinci.', 
     { 'entities': [(1,14,'WORK_OF_ART'),(53,70,'PERSON')]}
    ),
    ('"Romeo and Juliet" is a famous play written by William Shakespeare.',
     { 'entities': [(1,17,'WORK_OF_ART'),(48,67,'PERSON')]}
    ),
    
]

### Define Variable

In [67]:
model = None
output_dir=Path("/Users/khoadangnguyen/Desktop/spacy_implement")
n_iter=1200

### Load Model

In [68]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


### setup pipeline

In [69]:
if 'ner' not in nlp.pipe_names:
    #not ner = nlp.create_pipe('ner')
    #not nlp.add_pipe(ner, last=True)
    ner = nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

### Train recognizer with letter indices

In [70]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

from spacy.training.example import Example

#other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
#with nlp.disable_pipes(*other_pipes):  # only train NER
optimizer = nlp.begin_training()
for itn in range(n_iter):
    random.shuffle(TRAIN_DATA)
    losses = {}
    for text, annotations in tqdm(TRAIN_DATA):
        # create Example
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        # Update the model
        nlp.update(
            [example],
            drop=0.5,
            sgd=optimizer,
            losses=losses)
    print(losses)

  0%|          | 0/13 [00:00<?, ?it/s]

{'ner': 180.25179658271372}
{'ner': 103.62131993280491}


  0%|          | 0/13 [00:00<?, ?it/s]

{'ner': 62.964723104671975}
{'ner': 57.95479802338241}


  0%|          | 0/13 [00:00<?, ?it/s]

...

{'ner': 2.9836110498841206e-12}
{'ner': 1.2455729977573993e-06}


  0%|          | 0/13 [00:00<?, ?it/s]

{'ner': 1.4712178956196423e-09}
{'ner': 2.3942270740855006e-13}


100%|██████████| 13/13 [00:00<00:00, 91.76it/s]

{'ner': 3.5710545147692197e-07}


In [71]:
nlp.get_pipe('ner').labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'ORDINAL',
 'ORG',
 'PERSON',
 'PRODUCT',
 'WORK_OF_ART')

### Test Model

In [72]:
from spacy import displacy

for text, _ in TRAIN_DATA:
    doc = nlp(text)
    displacy.render(doc, style = "ent")
    #print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

### Saving locally

In [73]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)        

Saved model to /Users/khoadangnguyen/Desktop/spacy_implement


### Testing saved model

In [74]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TRAIN_DATA:
    doc = nlp2(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])


Loading from /Users/khoadangnguyen/Desktop/spacy_implement
Entities [('Anthony Son Bao', 'PERSON'), ('Don Khoa', 'PERSON')]
Tokens [('Anthony', 'PERSON', 3), ('Son', 'PERSON', 1), ('Bao', 'PERSON', 1), ('had', '', 2), ('4', '', 2), ('legs', '', 2), ('but', '', 2), ('lost', '', 2), ('one', '', 2), ('just', '', 2), ('now', '', 2), ('cause', '', 2), ('he', '', 2), ('touched', '', 2), ('Don', 'PERSON', 3), ('Khoa', 'PERSON', 1), ('.', '', 2)]
Entities [('The Mona Lisa', 'WORK_OF_ART'), ('Leonardo da Vinci', 'PERSON')]
Tokens [('"', '', 2), ('The', 'WORK_OF_ART', 3), ('Mona', 'WORK_OF_ART', 1), ('Lisa', 'WORK_OF_ART', 1), ('"', '', 2), ('is', '', 2), ('a', '', 2), ('renowned', '', 2), ('work', '', 2), ('of', '', 2), ('art', '', 2), ('painted', '', 2), ('by', '', 2), ('Leonardo', 'PERSON', 3), ('da', 'PERSON', 1), ('Vinci', 'PERSON', 1), ('.', '', 2)]
Entities [('Pep Guardiola', 'PERSON'), ('Treble', 'EVENT'), ('Man City', 'ORG')]
Tokens [('Pep', 'PERSON', 3), ('Guardiola', 'PERSON', 1), ('h

In [77]:
from spacy import displacy
TRAIN_DATA2 = [
    ('John Smith works for Acme Corporation, a multinational organization.', 
     { 'entities': [(0,10,'PERSON'), (21,37,'ORG')]}
    ),
    ('The Empire State Building is a famous skyscraper located in New York City.',
     { 'entities': [(4,25,'FAC'), (60,73,'GPE')] }
    ),
    ('Australia is a beautiful country known for its stunning landscapes and diverse wildlife.',
    ),
    ('Paris is often called the "City of Love" and is famous for its iconic landmarks such as the Eiffel Tower.',
     { 'entities': []}
    ),
    ('The iPhone X is a popular Apple product known for its sleek design and advanced features.',
     { 'entities': []}
    ),
    ('The World Cup is a major sporting event that takes place every four years, attracting teams from around the globe.',
     { 'entities': []}
    ),
    ('Dr. Elizabeth Johnson, a renowned scientist, works at NASA\'s Jet Propulsion Laboratory.',
     { 'entities': []}
    ),
    ('"The Starry Night," a masterpiece painted by Vincent van Gogh, is displayed at the Museum of Modern Art in New York City.',
    ),
    ('The price of a single Bitcoin reached $50,000, signaling a significant increase in its value.',
    ),
    ('The Taj Mahal, located in Agra, India, is an architectural marvel and a UNESCO World Heritage Site.',
    )
]

In [78]:
for text in TRAIN_DATA2:
    doc = nlp(text[0])
    displacy.render(doc, style = "ent")

In [18]:
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example

def my_evaluate(ner_model, TRAIN_DATA):
    scorer = Scorer()
    example = []
    for input_, annotations in TRAIN_DATA:
        pred = ner_model(input_)
        print(pred,annotations)
        temp = Example.from_dict(pred, dict.fromkeys(annotations))
        example.append(temp)
    scores = scorer.score(example)
    return scores

ner_model = spacy.load('en_core_web_sm') # for spaCy's pretrained use 'en_core_web_sm'
results = my_evaluate(ner_model, TRAIN_DATA)
print(results)

Anthony Son Bao had 4 legs but lost one just now cause he touched Don Khoa. {'entities': [(0, 15, 'PERSON'), (66, 74, 'PERSON')]}
Don Khoa loves Luyi Han more than anything {'entities': [(0, 8, 'PERSON'), (15, 23, 'PERSON')]}
Yesterday, on June 13, 2023, I attended the FIFA World Cup final in the Maracanã Stadium, Rio de Janeiro. The match was intense, and Brazil emerged as the champions, winning 2-1 against Argentina. {'entities': [(0, 9, 'DATE'), (14, 27, 'DATE'), (44, 64, 'EVENT'), (72, 88, 'FAC'), (90, 104, 'LOC'), (133, 124, 'FAC'), (133, 139, 'GPE'), (174, 174, 'CARDINAL'), (176, 177, 'CARDINAL'), (186, 196, 'GPE')]}
I bought 5 apples at the grocery store. {'entities': [(9, 10, 'CARDINAL'), (11, 17, 'PRODUCT')]}
The first time I heard about the Second Amendment was my first conversation with friends at the University of Rochester. {'entities': [(4, 9, 'ORDINAL'), (29, 49, 'LAW'), (92, 119, 'ORG')]}
I visited Paris, the city of love. {'entities': [(10, 15, 'GPE')]}
The Kevin Hart 

In [22]:
import pandas as pd
df = pd.read_table("ner_confusion.tbl",delim_whitespace=True,index_col=0)
df[:5]

FileNotFoundError: [Errno 2] No such file or directory: 'ner_confusion.tbl'

In [21]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_confusion_matrix(cm, normalize=True,):
    plt.figure(figsize=(10, 6))

    if normalize:
        cm = round(100*(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]),2)

    ax = sns.heatmap(cm, 
                    cmap='Blues', cbar_kws={"aspect": 7.5,"shrink":0.5},
                    annot=True, annot_kws = {"size":12},
                    fmt='.2f', linewidths=1, linecolor='white', square=True)

    ax.set_xlabel("Predicted", fontsize=14)
    ax.set_ylabel("Annotated", fontsize=14)
    ax.xaxis.tick_top()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    ax.xaxis.set_label_coords(0.5,-0.03)
    ax.yaxis.set_label_coords(-0.1,0.5)

    return ax